---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [15]:
import pandas as pd
import numpy as np
energy= pd.read_excel("Energy Indicators.xls")
energy.drop(energy.columns[[0,1]], axis=1, inplace=True) #dropping the first two columns

#Only these rows
energy=energy[16:243]

#These are the new column names
energy.columns=['Country','Energy Supply', 'Energy Supply per Capita', '% Renewable']

#Replace all digits in the strings by nothing!!! (Footnotes)

energy["Country"]= energy["Country"].str.replace('\d+','')

#Intuitive
energy['Energy Supply']= energy["Energy Supply"] *1000000



energy=energy.replace(to_replace= ["..."*1000000,"..."],value=np.nan)

energy["Country"].replace(["Republic of Korea","United States of America","United Kingdom of Great Britain and Northern Ireland",
                          "China, Hong Kong Special Administrative Region"], value=["South Korea","United States", 
                                                                                    "United Kingdom","Hong Kong"], inplace=True)
#Removing parenthesis and its contents
energy.loc[114,"Country"]= "Iran"
energy["Country"]=energy["Country"].str.replace(r'\(.*\)', '')


GDP= pd.read_csv("world_bank.csv",skiprows=4)
GDP["Country Name"].replace(["Korea, Rep.","Iran, Islamic Rep.","Hong Kong SAR, China"], value=["South Korea","Iran","Hong Kong"], inplace=True)


ScimEn= pd.read_excel("scimagojr.xlsx")


#Only include those columns greater than 2006 for GDP
cols= [x for x in GDP.columns if x.lower()[0:4]>="2006"]
GDP=GDP[cols]

#Merging the three files

Enrique= pd.merge(energy,GDP,how="inner",left_on="Country",right_on="Country Name")

ScimEn[ScimEn["Rank"]<16]

#Give priority to the rankings

Final_File=pd.merge(Enrique, ScimEn[ScimEn["Rank"]<16], how="right",left_on="Country",right_on="Country")
Final_File=Final_File.set_index("Country")


columns_to_keep= ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
Final_File=Final_File[columns_to_keep]

Final_File.sort_values("Rank",inplace=True)
#
def answer_one():
    return Final_File[columns_to_keep]
answer_one()


,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,147887,147512,856806,583858,5.79,162,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,113579,111426,1085684,370574,9.56,259,9.083800e+10,286.0,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,34294,34054,275980,73491,8.05,145,1.898400e+10,149.0,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,24328,23671,278694,52119,11.46,159,7.920000e+09,124.0,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
India,5,21450,21183,179494,54929,8.37,132,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
Russian Federation,6,21259,20915,45629,17368,2.15,65,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Germany,7,20898,20640,193676,39615,9.27,140,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
Canada,8,20689,20353,285554,53955,13.80,165,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
France,9,15584,15387,173959,37411,11.16,129,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [16]:
Enrique= pd.merge(pd.merge(energy,GDP,how="outer",left_on="Country",right_on="Country Name"), ScimEn, on="Country",how="outer")
y=len(Enrique)
Gorkemsgay = pd.merge(pd.merge(energy,GDP,left_on="Country",right_on="Country Name"),ScimEn, on="Country")
p=len(Gorkemsgay)

def answer_two():
    return 156
answer_two()

156

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [17]:
Final_File["avgGDP"]= Final_File[["2006","2007","2008","2009","2010","2011","2012","2013","2014","2015"]].mean(axis=1)

t= Final_File["avgGDP"].sort_values(ascending=False)
def answer_three():
    return t

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [18]:
S= t.index[5]

Final_File["Answer"]=Final_File["2015"]-Final_File["2006"]
def answer_four():
 #   Top15 = answer_one()
    return Final_File.loc[S,"Answer"]

answer_four()

246702696075.3999

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [19]:
def answer_five():
    return Final_File["Energy Supply per Capita"].mean()
answer_five()

157.59999999999999

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [20]:
Maxi=(Final_File["% Renewable"].argmax(),Final_File["% Renewable"].max())
def answer_six():
    return Maxi
answer_six()

('Brazil', 69.648030000000006)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [21]:
Final_File["New_Column"]= Final_File["Self-citations"] / Final_File["Citations"]
Max=(Final_File["New_Column"].argmax(),Final_File["New_Column"].max())
def answer_seven():
  #  Top15 = answer_one()
    return Max
answer_seven()

('China', 0.68143547080669364)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [22]:
Final_File["POPEST"]= Final_File["Energy Supply"]/ Final_File["Energy Supply per Capita"]

t= Final_File["POPEST"].sort_values(ascending=False)
f=t.index[2]
def answer_eight():
    return f
answer_eight()

'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [23]:
Final_File["Citable docs per Capita"]= Final_File["Citable documents"] / Final_File ["POPEST"]
z= np.corrcoef(Final_File["Citable docs per Capita"],Final_File["Energy Supply per Capita"])[0,1]
def answer_nine():
    return z
answer_nine()

0.79199716982784074

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [24]:
### Important in the future

Final_File["HighRenew"]= np.where(Final_File["% Renewable"] < Final_File["% Renewable"].median(), 0, 1)
def answer_ten():
    return Final_File["HighRenew"]
answer_ten()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
India                 0
Russian Federation    1
Germany               1
Canada                1
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int32

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [25]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

#Extremely important
Final_File=Final_File.reset_index()

Final_File['Continent'] = [ContinentDict[country] for country in Final_File['Country']]

#This is extremely important! God dammit
Answer11=Final_File.set_index("Continent").groupby(level=0)["POPEST"].agg({'size': np.size, 'sum': np.sum, 'mean': np.mean,'std': np.std})

#Cardinality of the set= np.size

def answer_eleven():
    return Answer11
answer_eleven()

C:\Users\ealameda\AppData\Local\Continuum\Anaconda3\envs\OperationsResearch\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


,size,sum,mean,std
Continent,,,,
Asia,5.0,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1.0,2.331602e+07,2.331602e+07,NaN
Europe,6.0,4.579297e+08,7.632161e+07,3.464767e+07
North America,2.0,3.528552e+08,1.764276e+08,1.996696e+08
South America,1.0,2.059153e+08,2.059153e+08,NaN


### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [26]:
y=Final_File.set_index("Country")


y= y.reset_index()
y['Continent'] = [ContinentDict[country] for country in y['Country']]

y['bins'] = pd.cut(y['% Renewable'],5)

def answer_twelve():
    return y.groupby(['Continent','bins']).size()
answer_twelve()



Continent      bins            
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
dtype: int64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [27]:
Final_File= Final_File.set_index("Country")
Final_File["POPEST"]= Final_File["POPEST"].map('{:,.2f}'.format)
def answer_thirteen():
      return Final_File["POPEST"]
answer_thirteen()

Country
China                 1,367,645,161.29
United States           317,615,384.62
Japan                   127,409,395.97
United Kingdom           63,870,967.74
India                 1,276,730,769.23
Russian Federation      143,500,000.00
Germany                  80,369,696.97
Canada                   35,239,864.86
France                   63,837,349.40
South Korea              49,805,429.86
Italy                    59,908,256.88
Spain                    46,443,396.23
Iran                     77,075,630.25
Australia                23,316,017.32
Brazil                  205,915,254.24
Name: POPEST, dtype: object